#  MAT-PROJECT03. Оптимизация Проект

![](data/dwight.png)

Нет-нет вам не показалось и это не Deja vu, это лишь проект по оптимизации, который очень ждал встречи с вами! Какой же дата-сайентист откажется от математической гимнастики?

В реальной жизни задачи оптимизации встречаются крайне часто: мы оптимизируем свое рабочее расписание, используя записную книжку, todo-планировщики и календари; владельцы бизнеса также постоянно решают задачи оптимизации в целях увеличения прибыли или роста доли на рынке, инженеры решают задачи оптимизации применительно к инженерным сооружениям или механизмам. Но сегодня мы будем будем решать задачи оптимизации иного порядка, а а если точнее - то мы закрепим сегодня методы оптимизации для машинного обучения.

> Напомним, что любой процесс разработки модели “переводится” на язык математики и компьютера следующим образом. Задаётся общий вид модели - это некоторая функция $f(w, x)$, где $x$ - признаки (факторы, фичи, регрессоры), а $w$ - параметры модели. Задача состоит в том, чтобы подобрать “оптимальным” образом. Оптимальность определяется функцией потерь.

Например, модель может быть линейной, тогда нужно найти оптимальные коэффициенты, на которые нужно умножать признаки, чтобы получить предсказание.

В данном проекте мы разберём некоторые способы численной оптимизации.

## Задание 1. Стационарные точки многочлена

Для разминки решим следующую задачу. Вам дан многочлен третьей степени от одной переменной: $y=ax^3 + bx^2 + cx + d$. Нужно найти стационарные точки: минимумы, максимумы и перегибы.

Напомним, стационарные точки - это такие точки, в которых производная равна нулю.

- Входные данные: коэффициенты *a, b, c, d*.
- Результат: напишите функцию `zero_grad(a, b, c, d)`, которая вернёт список вещественных стационарных точек соответствующего многочлена, упорядоченных по возрастанию.

Например, `zero_grad(1, 0, 0, 0)` должна вернуть `[0.0]` (x^3 имеет единственную стационарную точку - перегиб в (0.0). 

In [61]:
import math

def get_roots(a, b, c):
    d = b**2 - 4*a*c
    if d < 0:
        return []
    d = math.sqrt(d)
    r1 = (-b - d) / (2*a)
    r2 = (-b + d) / (2*a)
    if d == 0:
        return [r1]
    return [r1, r2]

In [62]:
import math

def zero_grad(a, b, c, d):
    return get_roots(3*a, 2*b, c)

In [63]:
zero_grad(1, 0, 0, 0)

[0.0]

In [64]:
zero_grad(1, 2, 1, 4)

[-1.0, -0.3333333333333333]

In [65]:
zero_grad(1/3, -1, 1, -10)

[1.0]

## Задание 2. Оптимизация функции двух переменных

Пусть теперь у нас есть функция $ g(x, y) = ax^2 + bxy + cy^2 + dx + ey + f $. Известно, что $a>0$, $c>0$, $b^2 - 4ac {\neq} 0$. Мы хотим найти минимум этой функции.

Из того, что $a>0$ и $c>0 $следует, что функция стремится в ${+\infty}$ при $x$ и $y$стремящихся в ${\infty}$. Для того, чтобы найти минимум такой функции, нужно приравнять градиент (частные производные по $x$ и $y$) к 0. После этого нужно просто решить систему уравнений. Проверьте, что у этой системы будет ровно одно решение (обратите внимание на условие $b^2 - 4ac {\neq} 0$).

- Входные данные: числа *a, b, c, d, e, f*
- Результат: напишите функцию `min_point(a, b, c, d, e, f)`, возвращающую кортеж из двух чисел $(x_0, y_0)$, при которых многочлен $g(x_0, y_0)$ принимает минимальное значение.

Например, `min_point(2, 1, 1, -5, -3, 57)` должна вернуть `(1, 1)`. 

**solution**

1. define gradient

$g'x = 2ax + by + d$

$g'y = 2cy + bx + e$

2. create system

$
\begin{cases}
   2ax + by + d = 0\\
   2cy + bx + e = 0
\end{cases}
$

3. define matrix form

$
\begin{bmatrix}
2a & b\\
b & 2c
\end{bmatrix}
\begin{bmatrix}
x\\
y
\end{bmatrix}
=
\begin{bmatrix}
-d\\
-e
\end{bmatrix}
$

4. here is solution

$
\begin{bmatrix}
x\\
y
\end{bmatrix}
=
\begin{bmatrix}
2a & b\\
b & 2c
\end{bmatrix}^{-1}
\begin{bmatrix}
-d\\
-e
\end{bmatrix}
$

In [78]:
import numpy as np

In [87]:
def min_point(a, b, c, d, e, f):
    a = np.array([
        [2*a,   b],
        [  b, 2*c],
    ])
    i = np.linalg.inv(a)
    v = np.array([[-d, -e]]).T
    x, y = i @ v
    return x[0], y[0]

In [88]:
min_point(2, 1, 1, -5, -3, 57)

(0.9999999999999998, 1.0)

## Задание 3. Градиентный спуск

В задачах выше функции были заданы в явном виде и была возможность найти их минимумы, решив уравнение или систему уравнений. При обучении более сложных моделей, например нейронных сетей, аналитическое решение не всегда возможно.

В таких случаях применяют градиентный спуск. Идея метода заключается в том, чтобы вычислять вектор частных производных функции и менять значения параметров на небольшой шаг в сторону, противоположную градиенту. Такой метод работает, т. к. функция наиболее быстро растёт в направлении градиента и убывает в обратном.

В этой задаче мы продолжим экспериментировать с многочленом второй степени от двух переменных - будем искать его минимум. На каждом шаге вам требуется вычислить градиент $g(x,y)$ и сместить точку $(x,y)$ на заданную длину *rate* в направлении, противоположном градиенту. Это значит, что вам надо вычислить градиент, затем сделать длину этого вектора равной *rate* (напомним, длина вектора равна $\sqrt{x^2+y^2}$), и потом вычесть полученный вектор из $(x,y)$.

- Входные данные: коэффициенты *a, b, c, d, e, f* многочлена $ (x, y) = ax^2 + bxy + cy^2 + dx + ey + f $ и начальная точка $(x_0, y_0)$.
- Результат: напишите функцию `gradient(a, b, c, d, e, f, x0, y0, rate)`, которая сделает один шаг градиентного спуска (сделает шаг длины *rate* в сторону, противоположную градиенту) и вернёт обновлённые $(x_0, y_0)$.

Пример: `gradient(2, 1, 1, -5, -3, 57, 0, 0, 1)` должна вернуть `(0.8574929257125441, 0.5144957554275265)`.

In [93]:
def gradient(a, b, c, d, e, f, x0, y0, rate):
    # calc gradient vector of function
    gx = 2*a*x0 + b*y0 + d
    gy = 2*c*y0 + b*x0 + e
    
    # length of gradient
    gl = math.sqrt(gx**2 + gy**2)

    # normalize gradient vector
    nx = gx / gl
    ny = gy / gl
    
    # setup velocity vector
    vx = nx * rate
    vy = ny * rate
    
    # calc next point
    x1 = x0 - vx
    y1 = y0 - vy
    
    return x1, y1

In [92]:
gradient(2, 1, 1, -5, -3, 57, 0, 0, 1)

(0.8574929257125441, 0.5144957554275265)

## Задание 4. Обратный гессиан

Для более быстрой сходимости численных методов часто используют метод Ньютона. Для его работы необходимо вычислить матрицу, обратную к матрице вторых производных (гессиан). В этой задаче необходимо сделать именно это. Из-за проблем с работой библиотеки numpy на платформе, реализуйте решение без ее использования.

- Входные данные: коэффициенты *a, b, c, d, e, f* многочлена второй степени от двух переменных $ (x, y) = ax^2 + bxy + cy^2 + dx + ey + f $.
- Результат: напишите функцию `hessian_inv(a, b, c, d, e, f)`, возвращающую матрицу вторых производных в виде двумерного *numpy-массива*.

Например, `hessian_inv(2, 1, 1, -5, -3, 57, 0, 0)` должна вернуть:
```
[[ 0.28571429, -0.14285714],
 [-0.14285714,  0.57142857]]
```

In [99]:
def hessian_inv(a, b, c, d, e, f):
    h = np.array([
        [2*a,   b],
        [  b, 2*c],
    ])
    return np.linalg.inv(h)

In [100]:
hessian_inv(2, 1, 1, -5, -3, 57)

array([[ 0.28571429, -0.14285714],
       [-0.14285714,  0.57142857]])

## Задание 5. Минимум неизвестной функции

Самый простой численный метод, который работает даже тогда, когда вид оптимизируемой функции не известен - метод “сетки”. Допустим, у нас есть непрерывная функция $f(x)$ и отрезок $[a,b]$. Необходимо найти минимум этой функции на данном отрезке.

Идея метода заключается в следующем. Возьмём несколько точек на отрезке, например, точки $t_1=a+(b-a)/3$
и $t_2=a+2*(b-a)/3$. Вычислим значения $f(a)$, $f(t_1)$, $f(t_2)$, $f(b)$ и выберем минимальное из них. Если минимум находится среди первых двух точек, то передвинем правый конец отрезка в третью точку (т. е., новый отрезок будет $[a,t_2]$), иначе передвинем левый конец (т. е., новый отрезок станет равен $[t_1,b]$). Несложно заметить, что на каждом шаге длина отрезка уменьшается на треть. Это значит, что после *n* шагов длина отрезка составит $(2/3)n$ от первоначального. Например, за 30 шагов мы уменьшим отрезок в 200 тыс раз.

У этого метода есть ограничение - он сработает плохо, если функция имеет много локальных минимумов на отрезке.

Итак, в этой задаче вам нужно найти такое число $x_0$ из отрезка $[a,b]$, при котором функция $f(x)$ принимает минимальное значение. Гарантируется, что данная функция имеет один минимум на отрезке $[a,b]$.

- Входные данные: функция **f**, два числа *a, b (a<b)*.
- Результат: напишите функцию `min_point(f, a, b)`, возвращающую единственное число `x0` (аргумент функции, при котором достигается минимум) с точностью до $10^{-3}$

Примечание: `f` - объект-функция, если её вызвать с единственным аргументом `x`, она вернёт значение в точке `x`. Например, `f((a+b)/2)` вернёт значение в середине отрезка `[a, b]`.

Пример результата: `min_point(lambda x: (x-1)**2, -10, 10)` должна вернуть число, близкое к 1.

In [132]:
def min_point(f, a, b):
    for i in range(30):
        t1 = a + (b-a)/3
        t2 = a + 2*(b-a)/3
        
        ma = min(f(a), f(t1))
        mb = min(f(t2), f(b))
                
        if ma < mb:
            b = t2
        else:
            a = t1
    return a + (b-a) * 0.5

In [133]:
min_point(lambda x: (x-1)**2, -10, 10)

0.9999785340460556